In [13]:
import matplotlib.pyplot as plt
import numpy as np
import random as rand
plt.style.use('seaborn')

In [123]:
def dudc(P,Q):
    
    def add(I, item):
        if item not in I: # if the interval is not already in the set I, we add it
            I.append(item)

    def check(p, a, b): # check if p if in the interval [a,b]
        return (a <= p and p <= b)

    P.sort()  # sort point in ascending order
    Q.sort()  # sort the interval in ascending order 
    nb_p = 0
   
    if len(Q) == 0 : 
        print("The set of intervals is empty ")
        return []
   
    elif len(Q)==1 : 
        for p in P : 
            if check(p[0], Q[0], Q[1]) : nb_p += 1
        if nb_p == len(P) : return Q
    
    else : 
        I = []
        j = 0
        for p in P : # for each point, we are looking for intervals that contains it
            if (len(I) != 0 and check(p[0], I[len(I)-1][0], I[len(I)-1][1])):
  
                nb_p += 1
            else:
                for i in range(j, len(Q)):

                    j += 1
                    if Q[i][0] <= p[0] and p[0] <= Q[i][1]:
                        if (i != len(Q)-1 and p[0] < Q[i+1][0]) or (i==len(Q)-1):
                            nb_p += 1
                            add(I, Q[i])
                            break
                    else:
                        print("We can't cover all the points")
                        print("For example :", p, "is not covered")
                        return []
        if nb_p == len(P) : return I

In [124]:
P = [(1.5,0), (2.5,0), (1.8,0), (0.3,0), (2.54,0),(0.79,0),(1.99,0)]
Q = [(1.2,1.6),(1.6,2),(1.8,2.2),(2.5,2.9),(0,0.4),(1.4,1.8),(0.01,0.41)]
print("Interval used = ", dudc(P,Q))

We can't cover all the points
For example : (0.79, 0) is not covered
Interval used =  []


In [118]:
P1 = [(0.5,0),(1.75,0),(1,0),(2,0)]
Q1 = [(0,1),(0.75,1.75),(1,2)]
print("Interval used = ", dudc(P1,Q1))

new Q [(0, 1), (0.75, 1.75), (1, 2)]
Interval used =  [(0, 1), (1, 2)]


In [ ]:
def variante :
    I = []
    j = 0
    for p in P : # for each point, we are looking for intervals that contains it
        if (len(I) != 0 and check(p[0], I[len(I)-1][0], I[len(I)-1][1])):
                nb_p += 1
        else: 
            for i in range(j, len(Q)):
                if Q[i][1] < p[0]:
                    j += 1
                else :
                    if Q[i][0] <= p[0]:
                        if (i != len(Q)-1 and p[0] < Q[i+1][0]) or (i==len(Q)-1):
                            nb_p += 1
                            add(I, Q[i])
                            j += 1
                            break 
                    else :
                        print("We can't cover all the point")
                        return []              
    if nb_p == len(P) : return I
    
else : 
        I = []
        for p in P : # for each point, we are looking for intervals that contains it
            for i in range(len(Q)):
                # we prefers an interval already stored in I 
                # since the intervals are sorted, if i==len(Q)-1
                # => there is no interval before this one which contains the point
                # so we only have to check if the point is in the interval
                if (Q[i] in I or i==len(Q)-1): 
                    if check(p[0], Q[i][0], Q[i][1]):
                        nb_p += 1
                        add(I, Q[i])
                        break
                elif p[0] <= Q[i][1] and p[0] < Q[i+1][0] : 
                    nb_p += 1 
                    add(I, Q[i])
                    break
        if nb_p == len(P) : return I
    

In [61]:
class Circle:
    
    def __init__(self, center,  radius=1):
        self.center = center
        self.radius = radius
        self.covered = None

    def cover_point(self, point):
        return (((self.center[0] - point[0]) ** 2 + (self.center[1] - point[1]) ** 2) ** 0.5 <= self.radius)

In [62]:
class Node(object):
    
    def __init__(self, parent, value, used_circles, name=(0,0)):
        self.parent = parent
        self.name = name
        self.value = value
        self.used_circles = used_circles
        self.child1 = None # take
        self.child2 = None # don't take
    
    def add_child(self, circle):
        self.child1 = Node(self, self.value, self.used_circles, (self.name[0]+1,1))
        used_circles = list(self.used_circles)
        used_circles.remove(circle)
        self.child2 = Node(self, self.value-1, used_circles, (self.name[0]+1,2))
    
    def coveredPoints(self, P):
        for i in self.used_circles :
            nb = 0
            for p in P:
                for circle in self.used_circles:
                    if circle.cover_point(p):
                        nb += 1
                        break
        return nb
            
class Tree:
    
    def __init__(self, root, P, Q):
        self.root = root
        self.P = P
        self.Q = Q
        self.best_sol = Q
    
    def isRealisable(self):
        if len(P) != self.root.coveredPoints(self.P): return False
        return True
    
    def isSatisfied(self, node):
        return ((node.coveredPoints(self.P) == len(P)) and node.value <= len(self.best_sol))
        

def createTree(P,Q):
    def count(disk, P):
        return sum([int(disk.cover_point(p)) for p in P])
    
    dictDisk = dict()
    for disk in Q :dictDisk[disk] = count(disk,P)
    circles = [item[0] for item in sorted(dictDisk.items() ,  key=lambda x: x[1], reverse=True)]
    i = 0
    tree = Tree(Node(None, len(Q), circles),P,Q)
    currentNode = tree.root
    if tree.isRealisable():
        for circle in circles : 
            currentNode.add_child(circle)
            if tree.isSatisfied(currentNode.child2):
                tree.best_sol = currentNode.child2.used_circles
                toVisit = currentNode.child2
            if tree.isSatisfied(currentNode.child1) :
                tree.best_sol = currentNode.child1.used_circles
                toVisit = currentNode.child1
            currentNode = toVisit
    return tree

In [63]:
P = [(1.5, 2.5), (3.5, 3.5), (4.5, 2.5), (4.5, 3.5), (5.5, 3.5), (5.5, 3.4), (1.5, 2.6), (2, 1)]
Q = [Circle((0, 0)), Circle((2, 2)), Circle((4.5, 2)), Circle((5, 3)), Circle((6, 4)), Circle((4,3))]

In [65]:
#tree = createTree(P,Q)
#toVisit = [tree.root]
#while toVisit != []:
#    current = toVisit[0]
#    print(current.name)
#    for circle in current.used_circles: print(circle.center, end = " ")
#    print()
#    if current.child1 != None :
#        toVisit.append(current.child1)
#    if current.child2 != None :
#        toVisit.append(current.child2)
#    toVisit.pop(0)